In [5]:
from zipfile import ZipFile 
import shutil
import os
import urllib.request
import pandas as pd


DATA_DIR = "data"

print("Downloading dataset...")
# download dataset and save it in a zip archive
url = "https://zenodo.org/record/6626316/files/Greek%20Parliament%20Proceedings%20Dataset_Support%20Files_Word%20Usage%20Change%20Computations.zip"
with urllib.request.urlopen(url) as f:
    zip_contents = f.read()

zip_path = os.path.join("data", "greek.zip")
with open(zip_path, "wb") as file:
    file.write(zip_contents)

# unzip saved archived
print("Unzipping...")
with ZipFile(zip_path, 'r') as zfile:
    zfile.extractall(path=DATA_DIR)

# organize zipped directory
old_dir = os.path.join(DATA_DIR, "Greek Parliament Proceedings Dataset_Support Files_Word Usage Change Computations")
new_dir = os.path.join(DATA_DIR, "greek")

# delete MACOSX directory
shutil.rmtree(os.path.join(DATA_DIR, "__MACOSX"))
# rename directory to a sensible name
os.rename(old_dir, new_dir) 


# load dataset
print("Loading...")
df = pd.read_csv(os.path.join(new_dir, "tell_all_cleaned.csv"), encoding="utf-8")
df 

Unzipping...
Loading...


,member_name,sitting_date,parliamentary_period,parliamentary_session,parliamentary_sitting,political_party,government,member_region,roles,member_gender,speaker_info,speech
0,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,παρακαλειται @sw γραμματεας βουλγαρακης @sw συ...
1,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,παρακαλειται @sw κυριος γραμματεας @sw συνοδευ...
2,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,κυριοι συναδελφοι παρακαλω @sw βουλη @sw εξουσ...
3,NaN,03/07/1989,period 5,session 1,sitting 1,βουλη,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],NaN,NaN,NaN,βουλευτης/ες,@sw @sw
4,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,προεδρευων,@sw βουλη παρεσχε @sw ζητηθεισα εξουσιοδοτηση....
...,...,...,...,...,...,...,...,...,...,...,...,...
1280913,κωνσταντινοπουλος κωνσταντινου οδυσσεας,24/07/2020,period 18 review 9,session 1,sitting 187,κινημα αλλαγης,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],αρκαδιας,['ε αντιπροεδρος βουλης(18/07/2019-28/07/2020)'],male,προεδρευων,κυριες @sw κυριοι συναδελφοι παρακαλω @sw σωμα...
1280914,NaN,24/07/2020,period 18 review 9,session 1,sitting 187,βουλη,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],NaN,NaN,NaN,βουλευτης/ες,@sw @sw
1280915,κωνσταντινοπουλος κωνσταντινου οδυσσεας,24/07/2020,period 18 review 9,session 1,sitting 187,κινημα αλλαγης,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],αρκαδιας,['ε αντιπροεδρος βουλης(18/07/2019-28/07/2020)'],male,προεδρευων,@sw σωμα παρεσχε @sw ζητηθεισα εξουσιοδοτηση κ...
1280916,NaN,24/07/2020,period 18 review 9,session 1,sitting 187,βουλη,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],NaN,NaN,NaN,βουλευτης/ες,@sw @sw


We replaced all references to political parties with the symbol “@” followed by an abbreviation of the
party name. We removed accents, strings with length less than 2 characters, all punctuation except
full stops, and replaced stopwords with “@sw”.

In [15]:
speech_df = df.loc[:, ["sitting_date", "speech"]].copy()

speech_df.sitting_date = pd.to_datetime(speech_df.sitting_date, dayfirst=True)
speech_df.set_index("sitting_date", inplace=True)
speech_df.sort_index(inplace=True) # required to avoid wrong slicing (and warnings)

speech_df = speech_df[~speech_df.speech.isnull()]

speech_df = speech_df.loc["2001-01-01":] 
speech_df

,speech
sitting_date,
2001-01-08,κυριες @sw κυριοι συναδελφοι αρχιζει @sw συνεδ...
2001-01-08,κυριες @sw κυριοι συναδελφοι @sw @sw τιμη @sw ...
2001-01-08,@sw @sw
2001-01-08,@sw σωμα ενεκρινε
2001-01-08,κυριοι συναδελφοι εισερχομαστε @sw ημερησια δι...
...,...
2020-07-24,κυριες @sw κυριοι συναδελφοι παρακαλω @sw σωμα...
2020-07-24,@sw @sw
2020-07-24,@sw σωμα παρεσχε @sw ζητηθεισα εξουσιοδοτηση κ...


In [19]:
from tqdm.notebook import tqdm_notebook


# enable progress bar functionality
tqdm_notebook().pandas()

0it [00:00, ?it/s]

In [21]:
# substantially faster than regex
def remove_placeholders(x: str) -> str:
    words = x.split()
    new_words = [word for word in words if not word.startswith("@")]
    return " ".join(new_words)

print("Removing placeholders...")
speech_df.speech = speech_df.speech.progress_apply(remove_placeholders)
speech_df = speech_df.loc[speech_df.speech.apply(lambda x: len(x.strip()) != 0)]
speech_df

Removing placeholders...


  0%|          | 0/836997 [00:00<?, ?it/s]

,speech
sitting_date,
2001-01-08,κυριες κυριοι συναδελφοι αρχιζει συνεδριαση ευ...
2001-01-08,κυριες κυριοι συναδελφοι τιμη ανακοινωσω σωμα ...
2001-01-08,σωμα ενεκρινε
2001-01-08,κυριοι συναδελφοι εισερχομαστε ημερησια διαταξ...
2001-01-08,κυριε προεδρε ευχηθω συναδελφους προσωπικα χρο...
...,...
2020-07-24,θεσεις κομματων αποτυπωθηκαν ψηφιση ηλεκτρονικ...
2020-07-24,ολοκληρωση ψηφοφοριας ηλεκτρονικο συστημα σχεδ...
2020-07-24,κυριες κυριοι συναδελφοι παρακαλω σωμα εξουσιο...
